In [1]:
import rasterio as rio
import earthpy.spatial as es
import os, sys
from glob import glob
import numpy as np
from rasterio.plot import show
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
from shapely.geometry import box
from rasterio.enums import Resampling
from rasterio.warp import reproject


In [2]:
# Find all sub folders 1 level down from parent_dir
parent_dir = 'Raster_data'
sub_dirs = [x[0] for x in os.walk(parent_dir)]
sub_dirs = sub_dirs[1:]
sub_dirs = [x for x in sub_dirs if os.path.basename(os.path.dirname(x)) == parent_dir]
sub_dirs = [x for x in sub_dirs if "LC08" in os.path.basename(x)]
# del sub_dirs[0]
print(sub_dirs)

['Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1']


In [3]:
import re

for sub_dir in sub_dirs:
    regex = 'B([0-7]|9|10|11)\.TIF$'
    band_paths = []
    all_paths = glob(os.path.join(sub_dir, '*'))
    for file in os.listdir(sub_dir):
        if re.search(regex, file):
            band_paths.append(os.path.join(sub_dir, file))
    
    print(band_paths)
    qa_path = glob(
        os.path.join(
            sub_dir,
            "*QA_PIXEL.TIF"
        )
    )
    paths_to_crop = band_paths + qa_path
    
    stack, metadata = es.stack(band_paths, out_path = sub_dir + '/Nagasaki_STACKED.tif')
    
    stack_mask = stack == 0
    stack_masked = np.ma.masked_array(stack, mask = stack_mask)
    
    with rio.open(sub_dir + '/Nagasaki_STACKED.tif') as src:
        bounds = src.bounds
        profile = src.profile
        src.close()
        
    target_shapefile = box(549000, 3599000, 635000, 3669000)
    
    band_paths = es.crop_all(
        paths_to_crop, sub_dir, [target_shapefile], overwrite=True
    )
    
    stack, metadata = es.stack(band_paths[:-1], out_path = sub_dir + '/Nagasaki_CROPPED_STACKED.tif')
    

['Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B1.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B10.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B11.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B2.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B3.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B4.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B5.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B6.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_20230509_02_T1\\LC08_L1TP_113037_20230502_20230509_02_T1_B7.TIF', 'Raster_data\\LC08_L1TP_113037_20230502_202